In [2]:
import pickle
import pandas as pd
import os
import networkx as nx


In [3]:
# names = ['../triplets_sci/lemmas_triplets.pickle',
#          '../triplets_sci/numbers_triplets.pickle',
#           ]

names = ['../triplets_env/lemmas_c_triplets.pickle',
         '../triplets_env/numbers_c_triplets.pickle',
          ]

total = []
for name in names:
    with open(name, 'rb') as f:
        ls = pickle.load(f)
        print(len(ls))
        total += ls

1741
1610


In [4]:
name = '../immutual_env_lemmas.pickle'
with open(name, 'rb') as f:
    ls = pickle.load(f)

path_env = '../gs_taxo/EN/environment_eurovoc_en.taxo'
G_env = nx.DiGraph()

with open(path_env, 'r') as f:
    for line in f:
        idx, hypo, hyper = line.split('\t')
        hyper = hyper.replace('\n', '')
        G_env.add_node(hypo)
        G_env.add_node(hyper)
        G_env.add_edge(hyper, hypo)

path_sci = '../gs_taxo/EN/environment_eurovoc_en.taxo'
G_sci = nx.DiGraph()

with open(path_sci, 'r') as f:
    for line in f:
        idx, hypo, hyper = line.split('\t')
        hyper = hyper.replace('\n', '')
        G_sci.add_node(hypo)
        G_sci.add_node(hyper)
        G_sci.add_edge(hyper, hypo)

In [5]:
def make_all_insertions(hyper, hypo, G):
    temp_ls = []
    for node in G.nodes():
        if (node != hyper) and (node != hypo) and (node != 'environment') and (node != 'science'):
            temp_ls.append((hyper, node, hypo))

    return temp_ls

def fill_ls(ls, G):

    cur_temp = []

    for child, parent in ls:
        cur_temp.extend(make_all_insertions(hyper=parent, hypo=child, G=G))

    return cur_temp

In [6]:
env_conflicts = fill_ls(ls, G_env)

In [7]:
total_insertions = list(set(env_conflicts + total))

In [8]:
def collect_insertions(total_ls):
    # transformed_term = (
    #         "hyperhypenym: "
    #         + clean["grandparents"]
    #         + " ("
    #         + elem["grandparent_def"]
    #         + "), hyponym: "
    #         + clean["children"]
    #         + " ("
    #         + elem["child_def"]
    #         + ") | hypernym:"
    #     )
    # return transformed_term, clean["parents"] + ","

    ls = []
    for triplet in set(total_ls):
        elem = {}
        elem["children"] = triplet[2]
        elem["parents"] = triplet[1]
        elem["grandparents"] = triplet[0]
        elem["case"] = "simple_triplet_grandparent"
        elem["grandparent_def"] = triplet[0]
        elem["child_def"] = triplet[2]
    
        ls.append(elem)

    return ls

In [9]:
insertions_env = collect_insertions(total_insertions)

In [19]:
out_path = '../data/env/eurovoc/raw_pairs/insertions_lemmas.pickle'
with open(out_path, 'wb') as f:
    pickle.dump(insertions_env, f)

In [75]:
def collect_mix_nodes(total_ls, out_path):

    ls = []
    for triplet in set(total_ls):
        elem = {}
        elem["children"] = triplet[2]
        elem["parents"] = [triplet[0], triplet[1]]
        elem["grandparents"] = None
        elem["case"] = "simple_triplet_2parent"
        elem["1parent_def"] = triplet[0]
        elem["2parent_def"] = triplet[1]
    
        ls.append(elem)

    with open(out_path, 'wb') as f:
        pickle.dump(ls, f)

In [76]:
collect_mix_nodes(total, '../data/env/eurovoc/raw_pairs/mix_nodes_lemmas.pickle')

In [77]:
with open('../data/env/eurovoc/raw_pairs/mix_nodes_lemmas.pickle', 'rb') as f:
    ls = pickle.load(f)

In [78]:
len(ls)

31066

In [28]:
ls = []
for triplet in set(total):
    elem = {}
    elem["children"] = triplet[2]
    elem["parents"] = [triplet[0], triplet[1]]
    elem["grandparents"] = None
    elem["case"] = "simple_triplet_2parent"
    elem["1parent_def"] = triplet[0]
    elem["2parent_def"] = triplet[1]

ls.append(elem)

In [29]:
elem

{'children': 'industrial sociology',
 'parents': ['industrial chemistry', 'ethnology'],
 'grandparents': None,
 'case': 'simple_triplet_2parent',
 '1parent_def': 'industrial chemistry',
 '2parent_def': 'ethnology'}

In [11]:

def clean_elem(elem, keys_to_remove_digits=["children"]):
    removes = set(keys_to_remove_digits)
    if not "changed" in elem.keys():
        for field in ["children", "parents", "grandparents", "brothers"]:
            if field in elem.keys():
                elem[field] = delete_techniqal(
                    elem[field], remove=True
                )  # (field in removes))
                elem["changed"] = True
    return elem


def delete_techniqal(elem, remove):
    if isinstance(elem, str):
        if ".n." in elem and remove:
            return elem.split(".")[0].replace("_", " ")
        else:
            return elem.replace("_", " ")

    elif isinstance(elem, list):
        new_words = []
        for word in elem:
            new_words.append(delete_techniqal(word, remove))
        return new_words
elem = insertions_env[0]
clean = clean_elem(elem, keys_to_remove_digits=["children"])
transformed_term = (
        "hyperhypenym: "
        + clean["grandparents"]
        + " ("
        + elem["grandparent_def"]
        + "), hyponym: "
        + clean["children"]
        + " ("
        + elem["child_def"]
        + ") | hypernym:"
    )

In [18]:
transformed_term

'hyperhypenym: hazardous waste (hazardous waste), hyponym: pollution of waterways (pollution of waterways) | hypernym:'

In [12]:
transformed_term
print(transformed_term)
s = transformed_term

hyperhypenym: hazardous waste (hazardous waste), hyponym: pollution of waterways (pollution of waterways) | hypernym:


In [13]:
term = s.split("|")[-2]
term = term.split(":")
term[1].split(' (')[0].strip()

'hazardous waste'

In [14]:
term[2].split('(')[0].strip()

'pollution of waterways'

In [15]:
def get_term(s, case):
    term = s.split("|")[-2]
    term = term.split(":")
    if (case == 'pred_hypernym') or (case == 'leaf_no_leafs'):
        return term[-1].strip()
    elif (case == 'simple_triplet_2parent') or (case == 'simple_triplet_grandparent'):
        first_parent = term[1].split(' (')[0].strip()
        second_parent = term[2].split('(')[0].strip()
        
        # in case of grandparent, it would be hyperhypernym_hyponym
        return first_parent + '_' + second_parent

In [17]:
get_term(transformed_term, case=elem['case'])

'hazardous waste_pollution of waterways'

In [41]:
s = ' industrial chemistry (industrial chemistry), second hypernym'

In [42]:
s.split('(')

[' industrial chemistry ', 'industrial chemistry), second hypernym']

In [47]:
with open('../data/env/eurovoc/raw_pairs/pred_hypernym_lemmas.pickle', 'rb') as f:
    ph = pickle.load(f)

In [48]:
ph[0]

{'children': 'climate change policy',
 'parents': 'adaptation to climate change',
 'grandparents': None,
 'case': 'predict_hypernym',
 'child_def': 'climate change policy'}

In [50]:
clean = clean_elem(ph[0], keys_to_remove_digits=["parents"])

transformed_term = (
    "hyponym: " + clean["children"] + " (" + ph[0]["child_def"] + ") | hypernym:"
)

In [52]:
get_term(transformed_term)

'climate change policy (climate change policy)'